## Import packages

In [19]:
import selenium
from selenium import webdriver

import shutil, time, os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from importlib import reload

import src
reload(src)

<module 'src' from 'C:\\Users\\Matth\\git\\DataAnalysisWorkbooks\\warcraftLogs\\src.py'>

## Configurations

In [7]:
# Set path variables
path_to_adblocker, path_to_download_dir = src.get_path_settings()

In [4]:
# Set character info
character_name = "Urbit"
character_server = "Benediction"
character_region = "US"

In [5]:
# Number of parses to scrape
N_parses = 2
retry_attempts = 5

# Enable/disable SMS
twilio = False

# Enable/disable verbose printouts (debugger)
verbose = True
verbose_rotation = False

## Navigate to character page

In [8]:
char_url, browser = src.load_individual_char_scraper(path_to_adblocker, character_name, character_server, character_region)

In [288]:
bosses = src.get_t6_bosses(browser)
bosses

["High Warlord Naj'entus",
 'Supremus',
 'Shade of Akama',
 'Teron Gorefiend',
 'Gurtogg Bloodboil',
 'Reliquary of Souls',
 'Mother Shahraz',
 'The Illidari Council',
 'Illidan Stormrage',
 'Rage Winterchill',
 'Anetheron',
 "Kaz'rogal",
 'Azgalor',
 'Archimonde']

In [9]:
# Alternatively specify specific bosses
bosses = ["High Warlord Naj'entus",
 'Supremus',
 'Shade of Akama',
 'Teron Gorefiend']

In [10]:
# Dictionary for browser to navigate to boss page URLs
boss_link_dict = {"High Warlord Naj'entus" : "#boss=601", "Supremus" : "#boss=602", "Shade of Akama" : "#boss=603", 
                  "Teron Gorefiend" : "#boss=604", "Gurtogg Bloodboil" : "#boss=605", "Reliquary of Souls" : "#boss=606", 
                  "Mother Shahraz" : "#boss=607", "The Illidari Council" : "#boss=608", "Illidan Stormrage" : "#boss=609", 
                  "Rage Winterchill" : "#boss=618", "Anetheron" : "#boss=619", "Kaz'rogal" : "#boss=620", 
                  "Azgalor" : "#boss=621", "Archimonde" : "#boss=622"}

## Main code loop

In [ ]:
def main():
    
    # Twilio is an SMS service that will send a text to your phone when the code finishes/crashes.
    # If you haven't setup twilio then set the flag to False in the Configuration cell.
    if twilio: 
        from twilio.rest import Client
        accountSID, authToken, myTwilioNumber, myCellPhone = src.get_twilio_info()
        twilioCli = Client(accountSID, authToken)
    
    
    retry_attempt = 0
    
    # The browser can crash at random due to the page loading too slowly. Rather than restarting the loop manually,
    # use 'retry_attempts' (default = 3) and the following while loop to restart automatically.
    while True: 
        
        #try:
    
        for boss in bosses:

            browser.get(f"https://classic.warcraftlogs.com/character/{character_region.lower()}/{character_server.lower()}/{character_name.lower()}{boss_link_dict[boss]}")
            time.sleep(1)

            # Pick out the top parse of this boss
            search = browser.find_elements_by_class_name("character-table-link")
            boss_page_url = browser.current_url

            print(f"Boss: {boss}")

            for i in range(N_parses):

                player_df = pd.DataFrame(pd.np.empty((0, 23)))
                player_df.columns = ["Name", "Server", "Date", "Kill time", "Rank", "nHealers", "Spriest?", "Innervate?", "Bloodlust?", "Nature's Grace?", "Power Infusion?", "LB_uptime", "HPS", "% LB (tick) HPS", "% LB (bloom) HPS", "% Rejuv HPS", "% Regrowth HPS", "% Swiftmend HPS", "Rotating on tank?", "Rotation 1", "% Rotation 1", "Rotation 2", "% Rotation 2"]

                try:
                    rank, date, boss_rank, boss_HPS, boss_killtime, HPS_link = src.get_boss_data_char_scraper(browser, i)

                except IndexError: 
                    continue

                search = browser.find_elements_by_class_name("character-table-link")

                if src.check_if_parse_already_recorded_char_scraper(i, browser, search, boss, character_name, character_server, character_region): 
                    print("Parse already recorded. Skipping")
                    browser.get(boss_page_url)
                    continue

                HPS_link.click()
                time.sleep(2)

                temp_url = browser.current_url

                boss_tanks = src.get_tanks(browser)
                nHealers = src.get_nHealers(browser)

                print(f"Tanks: {boss_tanks}")

                browser.get(temp_url)
                time.sleep(0.5)

                player_link = browser.find_element_by_link_text(character_name)
                player_link.click()
                time.sleep(0.5)

                # Scrape spell HPS
                LBtick_HPS, LBbloom_HPS, rejuv_HPS, regrowth_HPS, swiftmend_HPS, LB_uptime = src.get_spell_info(browser, boss_HPS)
                time.sleep(2)

                # Scrape mana sources
                spriest = src.check_spriest(browser)
                innervate, bloodlust, powerInfusion, naturesGrace = src.check_buffs(browser)

                # Download the cast-sequence CSV.
                src.download_csv(browser, temp_url, "filter-casts-tab", path_to_download_dir, "data/cast_sequence.csv")
                time.sleep(1)

                # Clean the csv
                df = src.clean_cast_sequence_csv()
                df = src.fix_cast_time(df)
                time.sleep(0.5)

                # Get the rotations
                rotation1, rotation1_percent, rotation2, rotation2_percent, rotating_on_tank = src.calculate_rotations(df, boss, boss_tanks, LB_uptime, verbose, verbose_rotation)
                print(f'Rotations: {rotation1} ({rotation1_percent}), {rotation2} ({rotation2_percent})')   

                # Add data to pandas dataframe
                to_append = [character_name, character_server + " " + character_region, date, boss_killtime, boss_rank, str(nHealers), spriest, innervate, bloodlust, powerInfusion, naturesGrace, LB_uptime, boss_HPS, LBtick_HPS, LBbloom_HPS, rejuv_HPS, regrowth_HPS, swiftmend_HPS, rotating_on_tank, rotation1, rotation1_percent, rotation2, rotation2_percent]
                src.export_to_excel(boss, to_append, player_df, character_name, 'character_data', False)

                os.remove(f"data/{boss.replace(' ', '')}_{character_name}.csv")
                os.remove(f"data/cast_sequence.csv")

                #Return to boss page
                browser.get(boss_page_url)
                time.sleep(1)  # Cannot be < 1s
                print('------')

            print(f"{boss} scraping complete.")
            print("---------------")

         # Return to character page
        browser.get(char_url)

        message = twilioCli.messages.create(body='Data scraping complete!', from_=myTwilioNumber, to=myCellPhone)
        break
            
        #except:
        #    retry_attempt += 1
        #    time.sleep(10)
        #    if retry_attempt == retry_attempts: 
        #        message = twilioCli.messages.create(body='Data scraping crashed', from_=myTwilioNumber, to=myCellPhone)
        #        break
    

if __name__ == "__main__":
    main()

Boss: High Warlord Naj'entus
Parse already recorded. Skipping
Tanks: ['Writz', 'Flebbis']
Lifebloom uptime %: 99.53%
2LB 3I 0RG 0.053
1LB 3I 0RG 0.368
1LB 2I 1RG 0.158
1LB 4I 0RG 0.211
0LB 1I 3RG 0.053
1LB 1I 2RG 0.053
0LB 4I 1RG 0.053
0LB 5I 0RG 0.053
Non-tank rotations %: 0.16
Rotating on tank?: Yes
Rotations: 1LB 3I 0RG (0.368), 1LB 4I 0RG (0.211)
------
High Warlord Naj'entus scraping complete.
---------------
Boss: Supremus
Tanks: ['Gankkd', 'Sanctusara', 'Flebbis']
Lifebloom uptime %: 100.00%
3LB 1I 0RG 0.267
2LB 2I 0RG 0.1
3LB 0I 0RG 0.033
3LB 0I 1RG 0.067
3LB 2I 0RG 0.033
2LB 2I 1RG 0.033
2LB 3I 0RG 0.133
0LB 3I 0RG 0.067
1LB 4I 0RG 0.067
1LB 3I 1RG 0.033
0LB 5I 0RG 0.067
2LB 1I 1RG 0.067
2LB 1I 0RG 0.033
Non-tank rotations %: 0.13
Rotating on tank?: Yes
Rotations: 3LB 1I 0RG (0.267), 2LB 3I 0RG (0.133)
------
Supremus scraping complete.
---------------
Boss: Shade of Akama
Tanks: ['Gankkd', 'Sanctusara', 'Flebbis']
Lifebloom uptime %: 100.00%
3LB 1I 0RG 0.267
2LB 2I 0RG 0.1
3L